In [5]:
col_names = [
    'dv', 
    'fv',
    'dv-fv'
]

In [6]:
import configparser
import warnings
import pandas as pd
from pymongo import MongoClient
import json
import requests
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore", category=DeprecationWarning)

config = configparser.ConfigParser()
config.read('config.ini')
host = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

client = MongoClient(host, int(port))
db = client['Media-cloud']

In [7]:
def get_text(url):
    if '/?nk=' in url:
        return ''
        
    text = ''
    try:
        html = requests.get(url).text
        soup = BeautifulSoup(html)

        for p in soup.find_all('p'):
            if p.parent.get('class') == ['section']:
                continue
            if p.get('class') == ['_2g0-m']:
                continue
            sentence = p.get_text().strip()
            if (not sentence.startswith('A NOTE ABOUT RELEVANT ADVERTISING') and
                not sentence.startswith('If you have inside knowledge') and
                not sentence.startswith('ABC teams share the story') and
                not sentence.startswith('Read about our editorial') and
                not sentence.startswith('Sign up to receive')
               ) and (
                sentence.endswith('.') or 
                sentence.endswith("'") or
                sentence.endswith('"') or
                sentence.endswith("?") or
                sentence.endswith("!") or
                sentence.endswith(";") or
                sentence.endswith(",") or
                sentence.endswith(")") or
                sentence.endswith(":")
            ):
                text += sentence + ' '
                
        return text.strip()
    except:
        return text.strip()

In [17]:
for col_name in col_names:
    counter = 0
    print('Now processing:', col_name+'.csv')
    
    df = pd.read_csv('input/'+col_name+'.csv')
    df['text'] = ''
    for i, row in df.iterrows():
        url = row['url']
        df.at[i, 'text'] = get_text(url)
        
        counter += 1
        if counter % 100 == 0:
            print(counter, 'records processed')
            
    col = db[col_name]
    records = json.loads(df.to_json(orient='records'))
    col.insert_many(records)
    
    print(col_name+'.csv finished\n--------------------\n')

Now processing: dv.csv
100 records processed
200 records processed
300 records processed
400 records processed
500 records processed
600 records processed
700 records processed
800 records processed
900 records processed
1000 records processed
1100 records processed
1200 records processed
1300 records processed
1400 records processed
1500 records processed
1600 records processed
1700 records processed
1800 records processed
1900 records processed
2000 records processed
2100 records processed
2200 records processed
2300 records processed
2400 records processed
2500 records processed
2600 records processed
2700 records processed
2800 records processed
2900 records processed
3000 records processed
3100 records processed
3200 records processed
3300 records processed
3400 records processed
3500 records processed
3600 records processed
3700 records processed
3800 records processed
3900 records processed
4000 records processed
4100 records processed
4200 records processed
4300 records process